In [ ]:
import os
import pandas as pd
import numpy as np
import osmnx as ox
from datetime import datetime 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from geopy.distance import distance
from shapely.geometry import Point
from shapely.geometry import LineString
import math
from pyproj import Geod
import geopandas as gpd
from tqdm import tqdm

### Finding mobile data points which are less than 2 km away from najafgarh station

In [ ]:
stations = pd.read_excel(r"E:\Thesis Codes\POI Data\stations_POI.xlsx")
stations.head()

In [ ]:
lat_njf, long_njf = stations.loc[20]['Latitude'], stations.loc[20]['Longitude']

In [ ]:
lat_njf, long_njf

In [ ]:
realtime = pd.read_csv(r"realtime_data_DEC.csv")
realtime2 = pd.read_csv(r"realtime_data_JAN.csv")
realtime.append(realtime2, ignore_index=True)
realtime.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
realtime = realtime[realtime.lat != 0]
realtime.reset_index(drop = True, inplace=True)

In [ ]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371

    # calculate the result
    distance = c * r
    return distance


In [ ]:
def find_distance(df, lat, long):
    d = []
    for i in range(len(df)):
        point1 = (df.loc[i,'long'], df.loc[i,'lat'])
        point2 = (long, lat)
        dist = haversine(lat_njf,long_njf,realtime.loc[i,'lat'],realtime.loc[i,'long'])
        d.append(dist)
    df['dist'] = d
    return df

In [ ]:
realtime = find_distance(realtime, lat_njf, long_njf)

In [ ]:
realtime = realtime[realtime.dist <= 2]
realtime.reset_index(drop=True, inplace=True)

### Joining statc PM 2.5 with the PM 2.5 values obtained from dynamic data points

In [ ]:
df_njf = pd.read_excel(r"C:\Users\vinee\Downloads\Thesis\PM_lodhi_road.xlsx")

In [ ]:
realtime.head()

In [ ]:
realtime.rename(columns = {'last_updated':'From Date'}, inplace = True)

In [ ]:
df_njf

In [ ]:
df_njf['From Date'] = df_njf['From Date'].apply(lambda x : datetime.strptime(x, '%d-%m-%Y %H:%M'))
realtime['From Date'] = realtime['From Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [ ]:
df = pd.merge(df_njf, realtime, on = 'From Date', how = 'inner')
df.head()

In [ ]:
df.rename(columns={'PM2.5_x' : 'PM2.5 static', 'PM2.5_y' : 'PM2.5 mobile'}, inplace=True)

In [ ]:
def find_midpoint(lat1, lng1, lat2, lng2):
    lat_mid = (lat1 + lat2) / 2
    long_mid = (lng1 + lng2) / 2
    return lat_mid, long_mid

### Checking mid point formula is giving correct results or not

In [ ]:
lat_mid, long_mid = find_midpoint(lat_njf, long_njf, df.loc[2, 'lat'], df.loc[2,'long'])
print(lat_mid, long_mid)

In [ ]:
import folium
m = folium.Map(location=[lat_njf, long_njf], zoom_start=13)
folium.Marker(location=[lat_njf, long_njf], popup='Point 1').add_to(m)
folium.Marker(location=[df.loc[2, 'lat'], df.loc[2,'long']], popup='Point 2').add_to(m)
folium.Marker(location=[lat_mid, long_mid], popup='Mid point 1').add_to(m)
m

### Code to add POIs(count), LandUse(area), water, buildings_area in our data

In [ ]:
# Code to filter dataframe on basis of the buffer formed by joining the line from the static monitor to the location of data point generated by mobile monitors
def buffer_point(df, lat, long, buffer_size):
    df['points'] = df.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
    count = 0
    for i in range(len(df)):
        point1 = (df.loc[i,'points'][0][0], df.loc[i,'points'][0][1])
        point2 = (long, lat)
        d = haversine(lat_njf, long_njf , df.loc[i,'points'][0][1] , df.loc[i,'points'][0][0])
        if d <= buffer_size:
            count += 1
    return count

# Code to filter dataframe on basis of the buffer formed by joining the line from the static monitor to the location of data point generated by mobile monitors
def buffer_polygon(df, point, buffer_size):
    df = df.to_crs({'init': 'epsg:3857'})
    point = gpd.GeoSeries(point, crs='epsg:4326').to_crs(epsg=3857).iloc[0]
    buffer = point.buffer(buffer_size)
    df = df[df.intersects(buffer)]
    if df.shape[0] == 0:
        return 0
    else:
        df['area_sq_m'] = df['geometry'].area
        return df['area_sq_m'].sum()/(10**6)


In [ ]:
def add_POI(df):
    poi = []
    for i in tqdm(range(len(df))):
        poi_gdf = gpd.read_file(r"C:\Users\vinee\Downloads\Thesis\SHP Files\poi_count.shp")
        lat_mid, long_mid = find_midpoint(lat_njf, long_njf, df.loc[i, 'lat'], df.loc[i,'long'])
        buffer_size = df.dist[i]/2
        poi.append(buffer_point(poi_gdf, lat_mid, long_mid, buffer_size))
    df['POI'] = poi

In [ ]:
add_POI(df)

In [ ]:
df

In [ ]:
# Code to convert a multipolygon in water shape file to polygon
def process(gdf):
    # Get the index of the row containing the MultiPolygon geometry
    idx = gdf.index[gdf.geometry.type == 'MultiPolygon']
    for i in idx:
        gdf = gdf.explode(index=[i]).drop(i)
        gdf = gdf.reset_index(drop=True)
    return gdf

def add_landuse(df):
    area = []
    for i in tqdm(range(len(df))):
        landuse_gdf = gpd.read_file(r"C:\Users\vinee\Downloads\Thesis\SHP Files\LandUse.shp")
        landuse_gdf = process(landuse_gdf)
        lat_mid, long_mid = find_midpoint(lat_njf, long_njf, df.loc[i, 'lat'], df.loc[i,'long'])
        buffer_size = df.dist[i]/2
        buffer_size = buffer_size*(10**3) #Converting buffer size in metres
        point = Point(long_mid,lat_mid)
        area.append(buffer_polygon(landuse_gdf, point, buffer_size))
    df['landuse(sq_km)'] = area

In [ ]:
add_landuse(df)

In [ ]:
def add_water(df):
    area = []
    for i in tqdm(range(len(df))):
        landuse_gdf = gpd.read_file(r"C:\Users\vinee\Downloads\Thesis\SHP Files\water.shp")
        gdf = process(landuse_gdf)
        lat_mid, long_mid = find_midpoint(lat_njf, long_njf, df.loc[i, 'lat'], df.loc[i,'long'])
        buffer_size = df.dist[i]/2
        buffer_size = buffer_size*(10**3) #Converting buffer size in metres
        point = Point(long_mid,lat_mid)
        try:
            area.append(buffer_polygon(gdf, point, buffer_size))
        except:
            area.append(0)
    df['water(sq_km)'] = area

In [ ]:
add_water(df)

In [ ]:
def add_buildings(df):
    area = []
    for i in tqdm(range(len(df))):
        landuse_gdf = gpd.read_file(r"C:\Users\vinee\Downloads\Thesis\SHP Files\buildings_area.shp")
        lat_mid, long_mid = find_midpoint(lat_njf, long_njf, df.loc[i, 'lat'], df.loc[i,'long'])
        buffer_size = df.dist[i]/2
        buffer_size = buffer_size*(10**3) #Converting buffer size in metres
        point = Point(long_mid,lat_mid)
        area.append(buffer_polygon(landuse_gdf, point, buffer_size))
    df['buildings(sq_km)'] = area

In [ ]:
add_buildings(df)

In [ ]:
df.to_csv(r'df_poi_land_buildings.csv')

### Code to plot the shapefile (grey color) and the common area colored blue

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Set the CRS of the landuse_gdf to EPSG:3857
landuse_gdf = gpd.read_file(r"C:\Users\vinee\Downloads\Thesis\SHP Files\LandUse.shp")
landuse_gdf = landuse_gdf.to_crs(epsg=3857)

# Create a shapely point object representing the point of interest
point = Point(long_njf, lat_njf)

# Set the CRS of the point to EPSG:3857
point = gpd.GeoSeries(point, crs='epsg:4326').to_crs(epsg=3857).iloc[0]

# Create a buffer around the point
buffer = point.buffer(1000)  # buffer radius in meters

# Select the landuse polygons that intersect with the buffer
intersected = landuse_gdf[landuse_gdf.intersects(buffer)]

# Create a new geopandas GeoDataFrame with the buffer polygon
buffer_gdf = gpd.GeoDataFrame(geometry=[buffer])

# Plot the buffer and the intersected polygons together
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_aspect('equal')
landuse_gdf.plot(ax=ax, color='lightgray', edgecolor='black')
intersected.plot(ax=ax, color='blue')
#buffer_gdf.plot(ax=ax, color='red')
plt.show()

In [ ]:
df.head(10)

In [ ]:
df_final = pd.read_csv(r"C:\Users\vinee\Downloads\Thesis\df_thesis.csv")
df_final['From Date'] = df_final['From Date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df_final['len_low_congestion'] = 0
df_final['len_medium_congestion'] = 0
df_final['len_high_congestion'] = 0

### Adding length of roads according to congestion factor

In [ ]:
def traffic(lat, long, buffer_radius):
    road_gdf = gpd.read_file(r"C:\Users\vinee\Downloads\Thesis\SHP Files\roads.shp")
    road_gdf = road_gdf.to_crs({'init': 'epsg:3857'})
    point = Point(long, lat)
    point = gpd.GeoSeries(point, crs='epsg:4326').to_crs(epsg=3857).iloc[0]
    buffer = point.buffer(buffer_radius)
    intersected = road_gdf[road_gdf.intersects(buffer)]
    intersected = intersected.reset_index(drop=True)
    return intersected

def date_time(date_obj):
    return datetime.strftime(date_obj, '%d-%b-%Y-%H-%M')

def to_date_time(x):
    return datetime.strptime(x,'%d-%b-%Y-%H')

In [ ]:
files = os.listdir(r"E:\HERE")
d = {'00':[], '15':[], '30':[], '45':[]}
for i in range(len(files)):
    dist = 1e3
    key = ''
    for j in d:
        diff = abs(int(files[i][-6:-4]) - int(j))
        if diff < dist:
            key = j
            dist = diff
    d[key].append(i)
l1 = d['00']
l2 = d['15']
l3 = d['30']
l4 = d['45']

lst_files = []
for i in l1:
    lst_files.append(files[i])
for i in l2:
    lst_files.append(files[i])
for i in l3:
    lst_files.append(files[i])
for i in l4:
    lst_files.append(files[i])

In [ ]:
df_final = df.copy()
for j in tqdm(range(len(df_final))):
    dt = df_final.loc[j,'From Date']
    lat_mid, long_mid = find_midpoint(lat_njf, long_njf, df_final.loc[j, 'lat'], df_final.loc[j,'long'])
    buffer_size = df_final.dist[j]/2
    buffer_size = buffer_size*(10**3)
    intersected = traffic(lat_mid, long_mid, buffer_size)
    for i in lst_files:
        congestion = []
        str_dt_time = i[:-7]
        dt_time = to_date_time(str_dt_time)
        if dt_time == dt:
            path = "E:\\HERE\\" + i
            df = pd.read_csv(path)
            all_roads = df['Names'].unique()
            for i in range(len(intersected)):
                if(intersected.loc[i,'name'] in all_roads):
                    ff = df[df['Names']==intersected.loc[i,'name']].ff.mean()
                    su = df[df['Names']==intersected.loc[i,'name']].su.mean()
                    if su == 0:
                        congestion.append(0)
                    else:
                        val = ff/su
                        if val > 1:
                            congestion.append(1)
                        else:
                            congestion.append(val)
                else:
                    congestion.append(0)
            intersected['congestion_factor'] = congestion
            congestion_low = intersected[intersected['congestion_factor'] <= 0.3]
            congestion_low = congestion_low[congestion_low.name.isna()]
            congestion_low = congestion_low.reset_index(drop=True)
            congestion_medium = intersected[(intersected['congestion_factor'] > 0.3) & (intersected['congestion_factor'] <= 0.7)]
            congestion_medium = congestion_medium.reset_index(drop=True)
            congestion_high = intersected[intersected['congestion_factor'] > 0.7]
            congestion_high = congestion_high.reset_index(drop=True)
            df_final.at[j,'len_low_congestion'] = congestion_low.geometry.length.sum()/(1000)
            df_final.at[j,'len_medium_congestion'] = congestion_medium.geometry.length.sum()/(1000)
            df_final.at[j,'len_high_congestion'] = congestion_high.geometry.length.sum()/(1000)
            break

In [ ]:
df

In [ ]:
df_final.to_csv('df4.csv')

In [ ]:
df_final[df_final.len_medium_congestion > 0]

In [ ]:
df_final

In [ ]:
df_final

In [ ]:
landuse_gdf = gpd.read_file(r"C:\Users\vinee\Downloads\Thesis\SHP Files\LandUse.shp")
landuse_gdf.head(20)